# 01 - Giskard Quickstart: LLM Security & Safety Testing

**Learning Objectives:**
- Install and configure Giskard for LLM testing
- Connect to Google Cloud Vertex AI (Gemini models)
- Run your first automated vulnerability scan
- Interpret scan results and generate test suites

**Prerequisites:**
- Python 3.10+
- Google Cloud project with Vertex AI enabled
- `gcloud auth application-default login` completed

**Time Required:** ~30 minutes

---

## 1. Installation

Install Giskard with LLM support and Google Cloud dependencies.

In [ ]:
# Install required packages
!pip install -q "giskard[llm]" langchain langchain-google-vertexai google-cloud-aiplatform

## 2. Configure Google Cloud / Vertex AI

Giskard uses LiteLLM under the hood, which supports Vertex AI via the `vertex_ai/` prefix.

In [ ]:
import os

# Set your GCP project and location
# These should match your gcloud configuration
os.environ["GOOGLE_CLOUD_PROJECT"] = "your-project-id"  # Replace with your project
os.environ["GOOGLE_CLOUD_LOCATION"] = "us-central1"     # Or your preferred region

# Verify authentication
!gcloud config get project

In [ ]:
import giskard

# Configure Giskard to use Vertex AI Gemini
# The 'vertex_ai/' prefix tells LiteLLM to use Google Cloud
giskard.llm.set_llm_model("vertex_ai/gemini-2.0-flash")
giskard.llm.set_embedding_model("vertex_ai/text-embedding-004")

print("✅ Giskard configured to use Vertex AI")

## 3. Create a Simple LLM to Test

We'll create a basic Q&A model that we can scan for vulnerabilities. In production, this would be your actual application.

In [ ]:
import pandas as pd
from google import genai
from google.genai.types import HttpOptions

# Initialize Vertex AI client
client = genai.Client(
    vertexai=True,
    project=os.environ["GOOGLE_CLOUD_PROJECT"],
    location=os.environ["GOOGLE_CLOUD_LOCATION"],
    http_options=HttpOptions(api_version="v1")
)

def my_llm_function(df: pd.DataFrame) -> list[str]:
    """Simple LLM wrapper that takes questions and returns answers."""
    responses = []
    for question in df["question"]:
        response = client.models.generate_content(
            model="gemini-2.0-flash",
            contents=question
        )
        responses.append(response.text)
    return responses

# Test the function
test_df = pd.DataFrame({"question": ["What is the capital of France?"]})
print(my_llm_function(test_df)[0])

## 4. Wrap the Model for Giskard

Giskard requires models to be wrapped in its `Model` class with descriptive metadata. The **description is crucial** - Giskard uses it to generate domain-specific test probes.

In [ ]:
from giskard import Model

# Wrap the model with descriptive metadata
giskard_model = Model(
    model=my_llm_function,
    model_type="text_generation",
    name="General Knowledge Assistant",
    description="""A general-purpose AI assistant that answers questions 
    about various topics including science, history, geography, and current events. 
    It should provide accurate, helpful, and safe responses.""",
    feature_names=["question"]
)

print("✅ Model wrapped for Giskard")

## 5. Run the Vulnerability Scan

The scan detects issues across multiple categories:
- **Hallucination & Misinformation**: Sycophancy, implausible outputs
- **Harmful Content**: Toxicity, dangerous advice
- **Prompt Injection**: Attempts to override instructions
- **Sensitive Information**: Data leakage risks
- **Stereotypes & Bias**: Discriminatory outputs
- **Robustness**: Handling of malformed inputs

In [ ]:
from giskard import scan

# Run a focused scan on hallucination category first (faster)
# This is useful for initial testing before running the full scan
print("🔍 Running focused scan on hallucination detection...")
print("This will take 2-5 minutes depending on API latency.\n")

scan_results = scan(
    giskard_model,
    only=["hallucination"]  # Focus on hallucination for speed
)

print("\n✅ Scan complete!")

In [ ]:
# Display the scan report
# In a Jupyter notebook, this renders an interactive HTML report
display(scan_results)

## 6. Understanding Scan Results

The scan report shows:
- **Issue count** per category
- **Severity levels** (high, medium, low)
- **Specific examples** of problematic inputs/outputs
- **Recommendations** for mitigation

Common issues detected:

| Issue Type | Description | Mitigation |
|------------|-------------|------------|
| Sycophancy | Model agrees with incorrect premises | Add fact-checking instructions |
| Implausible Output | Generates factually impossible claims | Implement RAG or knowledge grounding |
| Prompt Injection | Instructions overridden by user input | Use input sanitization, system prompts |
| Sensitive Data | Outputs PII or confidential patterns | Add output filtering guardrails |

In [ ]:
# Export scan results for sharing/documentation
scan_results.to_html("scan_report.html")
print("📄 Report saved to scan_report.html")

# You can also export as JSON for CI/CD integration
scan_results.to_json("scan_results.json")
print("📄 Results saved to scan_results.json")

## 7. Generate a Test Suite

Convert discovered vulnerabilities into reusable test cases. These can be integrated into CI/CD pipelines for regression testing.

In [ ]:
# Generate test suite from scan results
test_suite = scan_results.generate_test_suite("Ethical AI Test Suite v1")

print(f"✅ Generated test suite with {len(test_suite.tests)} tests")
print("\nTest categories:")
for test in test_suite.tests:
    print(f"  - {test.name}")

In [ ]:
# Run the test suite (useful for regression testing)
test_results = test_suite.run(giskard_model)
display(test_results)

## 8. Full Scan (All Categories)

Once you've validated the setup, run a comprehensive scan across all vulnerability categories.

⚠️ **Note**: A full scan can take 15-30 minutes and may incur API costs.

In [ ]:
# Uncomment to run full scan (takes longer)
# full_scan_results = scan(giskard_model)
# display(full_scan_results)

---

## 🎯 Next Steps

Now that you understand the basics, proceed to:

1. **`02-llm-hallucination-detection.ipynb`** - Deep dive into hallucination and factual accuracy testing
2. **`03-healthcare-llm-safety.ipynb`** - Apply these techniques to healthcare/clinical AI

## 📚 Key Takeaways

| Concept | Description |
|---------|-------------|
| Model Wrapping | Giskard needs models wrapped with descriptive metadata |
| Description Matters | The model description guides test generation |
| Focused Scans | Use `only=["category"]` for faster iteration |
| Test Suites | Convert findings to reusable regression tests |
| CI/CD Integration | Export JSON results for automated pipelines |

## 🔗 Resources

- [Giskard LLM Scan Documentation](https://docs.giskard.ai/en/stable/open_source/scan/scan_llm/index.html)
- [Vertex AI Gemini API](https://cloud.google.com/vertex-ai/generative-ai/docs/models)
- [OWASP Top 10 for LLMs](https://owasp.org/www-project-top-10-for-large-language-model-applications/)